# Initiilzation

In [1]:
import os
import itertools
import pandas as pd
import collections 
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
import pickle

# Create the Base lattice for the toy problem

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('Base_lattice_2.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)


# Create Enviornment Lattices

In [3]:
Bv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Gv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Yv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Rv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Vv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')

# Create Stencils

In [4]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

In [5]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

# Create the Graph 

In [6]:
# find the number of all voxels
vox_count = avail_lattice.size 

# initialize the adjacency matrix
adj_mtrx = np.zeros((vox_count,vox_count))

# extract the neighbourhood of all voxels
# all_vox_neighs = avail_lattice.find_neighbours(stencil)

# Finding the index of the available voxels in avail_lattice
avail_index = np.array(np.where(avail_lattice == 1)).T

# fill the adjacency matrix using the list of all neighbours
for vox_loc in avail_index:
    # find the 1D id
    vox_id = np.ravel_multi_index(vox_loc, avail_lattice.shape)
    # retrieve the list of neighbours of the voxel based on the stencil
    vox_neighs = avail_lattice.find_neighbours_masked(stencil_von_neumann, loc = vox_loc)
    # iterating over the neighbours
    for neigh in vox_neighs:
        # setting the entry to one
        adj_mtrx[vox_id, neigh] = 1.0

# construct the graph 
g = nx.from_numpy_array(adj_mtrx)

# Create Distance matrix 

In [7]:
#dist_mtrx = nx.floyd_warshall_numpy(g)

In [8]:
#dist_mtrx = pickle.dump( dist_mtrx, open( "dist_matrix_toy_problem.p", "wb" ) )

In [9]:
pickle_matrix = pickle.load( open( "dist_matrix_toy_problem.p", "rb" ) ) 

# Finding the center points of the divisions

In [10]:
index_lattice = np.copy(avail_lattice)


In [11]:
#slices= np.arange(1,20,3)
x_cordinate = np.array([2,8],dtype=int)
y_coordinate= np.arange(2,16,4,dtype=int)
print(x_cordinate)
indexes= []
for item in x_cordinate :
    for num in y_coordinate:
        a=item,num
        indexes.append(a)

[2 8]


In [12]:
for item in range(1,20,3):
    for num in indexes:
        selected_slice= index_lattice[item]
        selected_slice[num]=2
index_lattice_flat = index_lattice.flatten()    
indexing =list(np.where(index_lattice_flat ==2))
All_coordinates=[element for tupl in indexing for element in tupl]


In [13]:
print(avail_lattice[avail_lattice==1].size)

2394


In [14]:
## The number of voxels which each agent has to occupy There are five agents :[Blue,Green,Yellow,Red,Violet]
## The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1] 
mass_size = np.count_nonzero((avail_lattice==1)) 
B1 = int(mass_size / 5)
G1= int( mass_size / 5)
Y1=int( mass_size / 7.5)
R1= int(mass_size / 7.5)
V1= int(mass_size -(B1+G1+Y1+R1))
Num_of_voxels_lst = np.array([B1,G1,Y1,R1,V1])
Cube_root_list = Num_of_voxels_lst ** (1/3)
Radius_list = Cube_root_list.astype(int) 
Maximum_radius = int(Radius_list[np.argmax(Radius_list)])

np.sum(Num_of_voxels_lst)

2394

In [15]:
Num_of_voxels_lst

array([478, 478, 319, 319, 800])

# Compute the permutations and filter valid ones

In [16]:
pickle_matrix[35][22]

6.0

In [17]:
all_permutations_blue = (list(itertools.permutations(All_coordinates, 2)))

In [18]:
Blue_valid_perms_first=[]
for perm in all_permutations_blue:
    if pickle_matrix [perm[0]][perm[1]] == 7 :
        Blue_valid_perms_first.append(perm)

In [19]:
Blue_valid_perms_second =[]
for perm in Blue_valid_perms_first :
    for index in All_coordinates :
        if index != perm[1] and index != perm[0]:
            a=(perm[0],perm[1],index)
            Blue_valid_perms_second.append(a)


In [20]:
Blue_valid_perms_third =[]
for perm in Blue_valid_perms_second :
        for index in All_coordinates :
            if index != perm[0] and index != perm[1] and index != perm[2] :
                if pickle_matrix [perm[0]][index] >= 10 :
                    a=(perm[0],perm[1],perm[2],index)
                    Blue_valid_perms_third.append(a)

In [21]:
'''
Blue_valid_perms_fourth =[]
for perm in Blue_valid_perms_third :
        for index in All_coordinates :
            if index != perm[0] and index != perm[1] and index != perm[2] and index != perm[3] :
                if pickle_matrix [perm[0]][index] >= 12 : 
                    if pickle_matrix [perm[1]][index] == 9 :
                        if pickle_matrix [perm[2]][index] >= 11 :
                            a=(perm[0],perm[1],perm[2],perm[3],index)
                            Blue_valid_perms_fourth.append(a)
'''

'\nBlue_valid_perms_fourth =[]\nfor perm in Blue_valid_perms_third :\n        for index in All_coordinates :\n            if index != perm[0] and index != perm[1] and index != perm[2] and index != perm[3] :\n                if pickle_matrix [perm[0]][index] >= 12 : \n                    if pickle_matrix [perm[1]][index] == 9 :\n                        if pickle_matrix [perm[2]][index] >= 11 :\n                            a=(perm[0],perm[1],perm[2],perm[3],index)\n                            Blue_valid_perms_fourth.append(a)\n'

In [22]:
#all_valid_permutations = pickle.dump( Blue_valid_perms_fourth, open( "all_valid_permutations_toy_problem.p", "wb" ) )

In [23]:
pickle_permutations = pickle.load( open( "all_valid_permutations_toy_problem.p", "rb" ) ) 

In [24]:
Final_permutations = pickle_permutations

# Initilize agent class 

In [25]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
      
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
    def argmax_occupy_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
   
        
        
    def one_neighbour_occupy_cubish_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = neighs_availibility_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_cubish.append(neighs)

        neighs_flatten = np.array(env.neigh_cubish).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
                
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        
        #neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)

       # print(neighs_unq_base_value) 
        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )       
            
        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
        #print(self.origin)
        
        

# Enviornment Class

In [26]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_cubish = []
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
    
    def argmax_occupy_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.argmax_occupy_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
      
    def one_neighbour_occupy_cubish_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

# Initialize agents

In [27]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    agn_num = 1
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

def initialize_agents_fixed_origin (stencil,avail_lattice,origin):
    #finding origin 
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    agn_origins = np.unravel_index(origin,avail_lattice.shape)
    select_id = origin
    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

In [28]:
len(Final_permutations)

288088

# Blue function max simulation

In [29]:
env_availability_viz_blue= []
env_availability_score= []
for item in All_coordinates:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item)
    
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
    env_B = {"availibility": occ_lattice_sim,"enviornment": Bv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_B, Agent_one,B1,"one_neighbour_occupy_cubish_behaviour")
   
    env_list =[env_1]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_blue.append(e.availibility-1)
    env_availability_score.append(np.sum(e.value[e.availibility == item]))  

In [30]:
top_five_blue = sorted( [(x,i) for (i,x) in enumerate(env_availability_score)], reverse=True )[:15] 
top_five_blue_origins = []
for item in top_five_blue:
    top_five_blue_origins.append(All_coordinates[item[1]])
top_five_blue_origins

[3386,
 3482,
 2854,
 1278,
 3382,
 2858,
 846,
 318,
 842,
 750,
 746,
 314,
 222,
 218,
 3486]

# Green function max simulation

In [31]:
env_availability_viz_green= []
env_availability_score= []
for item in All_coordinates:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item)
    
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
    env_G = {"availibility": occ_lattice_sim,"enviornment": Gv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_G, Agent_one,G1,"one_neighbour_occupy_cubish_behaviour")
   
    env_list =[env_1]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_blue.append(e.availibility-1)
    env_availability_score.append(np.sum(e.value[e.availibility == item]))  

In [32]:
top_five_green = sorted( [(x,i) for (i,x) in enumerate(env_availability_score)], reverse=True )[:15] 
top_five_green_origins = []
for item in top_five_green:
    top_five_green_origins.append(All_coordinates[item[1]])
top_five_green_origins    

[1890,
 2418,
 3474,
 1270,
 1266,
 1362,
 3390,
 1794,
 1370,
 1278,
 1274,
 3382,
 1798,
 834,
 742]

# Yellow function max simulation

In [33]:
env_availability_viz_yellow= []
env_availability_score= []
for item in All_coordinates:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item)
    
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
    env_Y = {"availibility": occ_lattice_sim,"enviornment": Yv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_Y, Agent_one,Y1,"one_neighbour_occupy_cubish_behaviour")
   
    env_list =[env_1]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_yellow.append(e.availibility-1)
    env_availability_score.append(np.sum(e.value[e.availibility == item]))  

In [34]:
top_five_yellow = sorted( [(x,i) for (i,x) in enumerate(env_availability_score)], reverse=True )[:15] 
top_five_yellow_origins = []
for item in top_five_yellow:
    top_five_yellow_origins.append(All_coordinates[item[1]])
top_five_yellow_origins    

[3482,
 2958,
 3478,
 2430,
 1890,
 1362,
 834,
 2950,
 2862,
 2954,
 2330,
 1802,
 2426,
 1902,
 2418]

# Red function max simulation

In [35]:
env_availability_viz_red= []
env_availability_score= []
for item in All_coordinates:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item)
    
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
    env_R = {"availibility": occ_lattice_sim,"enviornment": Rv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_R, Agent_one,R1,"one_neighbour_occupy_cubish_behaviour")
   
    env_list =[env_1]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_red.append(e.availibility-1)
    env_availability_score.append(np.sum(e.value[e.availibility == item]))  

In [36]:
top_five_red = sorted( [(x,i) for (i,x) in enumerate(env_availability_score)], reverse=True )[:15] 
top_five_red_origins = []
for item in top_five_red:
    top_five_red_origins.append(All_coordinates[item[1]])
top_five_red_origins  

[3378,
 214,
 2850,
 1902,
 838,
 310,
 306,
 222,
 218,
 210,
 1362,
 1266,
 3382,
 2334,
 834]

# Violet function max simulation

In [37]:
env_availability_viz_violet= []
env_availability_score= []
for item in All_coordinates:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item)
    
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
    env_V = {"availibility": occ_lattice_sim,"enviornment": Vv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_V, Agent_one,V1,"one_neighbour_occupy_cubish_behaviour")
   
    env_list =[env_1]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_violet.append(e.availibility-1)
    env_availability_score.append(np.sum(e.value[e.availibility == item]))  

# Violet function Vizualization

In [38]:
p = pv.Plotter(notebook=True)

base_lattice = env_availability_viz_violet[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
#pickle_top_500_tglattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = env_availability_viz_violet[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.0, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = len(env_availability_viz_violet)
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(125.69980502474405, 110.69980502474405, 118.19980502474405),
 (30.0, 15.0, 22.5),
 (0.0, 0.0, 1.0)]

In [39]:
env_availability_viz_violet[1]

lattice([[[ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
  

In [40]:
top_five_violet = sorted( [(x,i) for (i,x) in enumerate(env_availability_score)], reverse=True )[:15] 
top_five_violet_origins = []
for item in top_five_violet:
    top_five_violet_origins.append(All_coordinates[item[1]])
top_five_violet_origins  

[1798,
 1270,
 1266,
 1362,
 1794,
 1802,
 1370,
 1366,
 1274,
 1806,
 1278,
 2422,
 846,
 842,
 838]

# Top Positions all colours

In [41]:
Top_positions = []
for b,g,y,r,v in zip(top_five_blue_origins,top_five_green_origins,top_five_yellow_origins,top_five_red_origins,top_five_violet_origins):
    Top_positions.append((b,g,y,r,v))

final_positions=[]

for elem in Top_positions: 
    if elem in Final_permutations:
        final_positions.append(elem)
    else:
        print("false")
final_positions


false
false
false
false
false
false
false
false
false
false
false
false
false
false
false


[]

# Top positions of max impact function ( Biggest number of voxels)

In [42]:
Top_Violet=[]
for perm in Final_permutations:
    if top_five_violet_origins[0] == perm[4]:
                Top_Violet.append(perm)
type(Top_Violet[0])    

tuple

In [43]:
flat_list= np.array(Top_Violet).flatten()
set_list =set(flat_list)
len(set_list)

56

# Top 500 Simulations

In [44]:
'''
env_availability_viz_top_500 = []

for item in Top_Violet[0:500]:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item[0])
    Agent_two=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item[1])
    Agent_three=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item[2])
    Agent_four=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item[3])
    Agent_five=initialize_agents_fixed_origin (stencil_cuboid,avail_lattice,item[4])
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)
    env_B = {"availibility": occ_lattice_sim,"enviornment": Bv}
    env_G = {"availibility": occ_lattice_sim,"enviornment": Gv}
    env_R = {"availibility": occ_lattice_sim,"enviornment": Rv}
    env_Y = {"availibility": occ_lattice_sim,"enviornment": Yv}
    env_V = {"availibility": occ_lattice_sim,"enviornment": Vv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_B, Agent_one,B1,"one_neighbour_occupy_cubish_behaviour")
    env_2 = environment(env_G, Agent_two,G1,"one_neighbour_occupy_cubish_behaviour")
    env_3 = environment(env_Y, Agent_three,Y1,"one_neighbour_occupy_cubish_behaviour")
    env_4 = environment(env_R, Agent_four,R1,"one_neighbour_occupy_cubish_behaviour")
    env_5 = environment(env_V, Agent_five,V1,"one_neighbour_occupy_cubish_behaviour")
    
    
    env_list =[env_2,env_3,env_4,env_5]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_top_500.append(e.availibility)    
'''

In [45]:
env_availability_viz_top_500[0]


lattice([[[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0,    0,  

In [46]:
top_500 = pickle.dump( env_availability_viz_top_500, open( "top_500_lattices.p", "wb" ) )
pickle_top_500 = pickle.load( open( "top_500_lattices.p", "rb" ) ) 
pickle_top_500_array = np.array(pickle_top_500)
pickle_top_500_tglattice=tg.to_lattice(np.copy(pickle_top_500_array), pickle_top_500_array.shape)
pickle_top_500_tglattice.shape

(500, 21, 11, 16)

In [47]:
pickle_top_500_tglattice[0][pickle_top_500_tglattice[0]==834]

lattice([834])

In [48]:
abc= pickle_top_500_tglattice.reshape(500*21,11,16)
abc[1]

lattice([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0],
         [  0, 210, 210, 210, 210, 214, 214, 214, 214, 214, 214, 214,
          214, 214, 214,   0],
         [  0, 210, 210, 210, 210, 214, 214, 214, 214, 214, 214, 214,
          214, 214, 214,   0],
         [  0, 210, 210, 210, 210, 214, 214, 214, 214, 214, 214, 214,
          214, 214, 214,   0],
         [  0, 210, 210, 210, 214, 214, 214, 214, 214, 214, 214, 214,
          214, 214, 214,   0],
         [  0, 210, 210, 210, 214, 214, 214, 214, 214, 214, 214, 214,
          214, 214, 214,   0],
         [  0, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214,
          214, 214, 214,   0],
         [  0, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214,
          214, 214, 214,   0],
         [  0, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214,
          214, 214, 214,   0],
         [  0, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214,
          214, 214, 2

# Save to Csv

In [49]:

for i, lattice in enumerate(env_availability_viz_top_500):
    csv_path = os.path.relpath('csv/abm_f_'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)


# Viz Top 500

In [50]:
p = pv.Plotter(notebook=True)

base_lattice = env_availability_viz_top_500[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
#base_env_availability_viz_top_500[0].fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = env_availability_viz_top_500[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.0, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 500
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(125.69980502474405, 110.69980502474405, 118.19980502474405),
 (30.0, 15.0, 22.5),
 (0.0, 0.0, 1.0)]

# Calculate the score for each simulation

In [51]:
for lattice,index in zip (env_availability_viz_top_500,Top_Violet[0:500]):
    lattice_flat = lattice.flatten()
    B= np.sum(Bv.flatten()[lattice_flat == index[0]])
    G= np.sum(Gv.flatten()[lattice_flat == index[1]])
    Y= np.sum(Yv.flatten()[lattice_flat == index[2]])
    R= np.sum(Rv.flatten()[lattice_flat == index[3]])
    V= np.sum(Vv.flatten()[lattice_flat == index[4]])
    Val = B+G+Y+R+V
    #print(Val)